# DataBase Preview
This document analizes the DataBase "raw_data_project_m1.db" included in the folder data. 
The objetive is previsualize the data that are collected in the database

Fist of all, the libraries needed are imported.

In [126]:
import pandas as pd
import sqlalchemy
from functools import reduce
import json
import requests
import numpy as np


# AQUISITION AND CLEANING DATA

A connection to the datbase is created, called engine.

In [127]:
from sqlalchemy import create_engine
data_path = '../data/raw_data_project_m1.db'
connection = f'sqlite:///{data_path}'

engine = create_engine(connection)

Let's see what tables are included in the database.

In [128]:
db_raw = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)
display(db_raw)

,name
0,personal_info
1,country_info
2,career_info
3,poll_info


We can get a list of the name of tables included in the database

In [129]:
db_raw_list = db_raw.name.tolist()
print(db_raw_list)

['personal_info', 'country_info', 'career_info', 'poll_info']


Let's see what is into each dataframe

Each daraframe consists of related data in a common column 'uuid'

In [131]:
# personal_info table
df_personal_info = pd.read_sql_query("select * from personal_info", engine)
display(df_personal_info)
df_personal_info.dtypes

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Fem,yES,40_65
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,37 years old,FeMale,nO,26_39
9645,39f989f0-db52-0133-8482-0a81e8b09a82,53 years old,Male,yES,40_65
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,1992,male,NO,juvenile
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,47 years old,male,yES,40_65


uuid                object
age                 object
gender              object
dem_has_children    object
age_group           object
dtype: object

The database consists of four tables with different data. A visualization of each one is performed, in order to see the columns, type of data and number of rows.

The column "gender" is important for the analysis, thus let's see it's unique values.

In [132]:
df_personal_info["gender"].unique()

array(['male', 'Male', 'Fem', 'FeMale', 'female'], dtype=object)

This data should be homogeneos, thus they all are converted in "Male" or "Female"

In [133]:
def gender(column_g):
    if column_g.startswith('F' or 'f'):
        return 'Female'
    else:
        return 'Male'
    
df_personal_info['gender'] = df_personal_info.apply(lambda x: gender( x['gender']), axis=1 )
display(df_personal_info.head())

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,Male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,Male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,Male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Female,yES,40_65


In [134]:
# country_info table
df_country_info = pd.read_sql_query("select * from country_info", engine)
display(df_country_info)
df_country_info.dtypes

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city
...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,Non-Rural
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,city


uuid            object
country_code    object
rural           object
dtype: object

This table contains the column 'country_code'. These codes will be replaced by their respective country, getting the data from Eurostat website (web scraping). So let's see which codes are in the column (uniques values).

In [135]:
df_country_info["country_code"].unique()

array(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR',
       'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL',
       'PL', 'PT', 'RO', 'SE', 'SI', 'SK'], dtype=object)

In [136]:
# career_info table
df_career_info = pd.read_sql_query("select * from career_info", engine)
display(df_career_info)
df_career_info.dtypes

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a


uuid                   object
dem_education_level    object
dem_full_time_job      object
normalized_job_code    object
dtype: object

This table contains the column 'normalized_job_code'. These codes will be replaced by their respective job name, getting the data using the API from the Open Skills Project. So let's see which codes are in the column (uniques values).

In [137]:
job_codes = df_career_info["normalized_job_code"].unique()
list_job_codes = job_codes.tolist()
print(list_job_codes)

[None, '861a9b9151e11362eb3c77ca914172d0', '049a3f3a2b5f85cb2971ba77ad66e10c', 'f4b2fb1aa40f661488e2782b6d57ad2f', '27af8700f5577cec835acee2cb90a2ff', 'c1b670eba9ccb65e7c99f7da116d5b9c', 'd71c4164c99a5f9bca773e755a72f40f', '2ee80cf30e2483dfe0947adc0428f8c5', '897ef18a9321616495831a4b4b0c1bb2', 'a003702c0cea282e17967772dd83de5e', '04c82eae29e1c4a30eac0710b74ab276', 'fcaf4aac737e53ad24b9f32f08b8740f', '3833bd699e7afe4665b595f282da0387', '19e384e977796c25780d8c28b12089c4', '474b962c13632a04e42a546be4f11dbb', 'dac2e921c1f887d9efc4c06fa33c1397', '996da29c84f5eed00da40c230f4c4078', 'cc2b26cc436bea36bda3bdd7367c411a', '25fe195cbf915c0824fa6c44e6d0008c', '76385cb24e831b3364279b9de5641d8f', '16b67d7f57698455aefbc1ae1b358b50', 'b24ece3300d2f8ab34f9553e531183d6', '5db2c8d6539f983e1ca3d7c5b939e190', '36675c937946c8a4fc5de7d0b4c850de', '847165cfda6b1dc82ae22b967da8af2f', '2ca138b991c50dc6696238e5ad800142', '775190277a849cba701b306a7b374c0a', '022d0e3a40aa3e29bca6851542b27842', '375017ea3a1a97f5ea67

In [ ]:
The codes have been included in a list, to use it later. The first element (None), is deleted to avoid troubles.

In [154]:
#del list_job_codes[0]
list_job_codes = [x for x in list_job_codes if x is not None]
#list_job_codes = list_job_codes.remove('None')
len(list_job_codes)

156

In [139]:
# poll_info table
df_poll_info = pd.read_sql_query("select * from poll_info", engine)
display(df_poll_info)
df_poll_info.dtypes

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


uuid                                                   object
question_bbi_2016wave4_basicincome_awareness           object
question_bbi_2016wave4_basicincome_vote                object
question_bbi_2016wave4_basicincome_effect              object
question_bbi_2016wave4_basicincome_argumentsfor        object
question_bbi_2016wave4_basicincome_argumentsagainst    object
dtype: object

The column uuid is common in the four tables. They can be joined together in an unique table by that column.

In [140]:
tables_to_join = [df_personal_info, df_country_info, df_career_info, df_poll_info]
jobs_complete = reduce(lambda left,right: pd.merge(left,right,on='uuid'), tables_to_join)
display(jobs_complete.head())

,uuid,age,gender,dem_has_children,age_group,country_code,rural,dem_education_level,dem_full_time_job,normalized_job_code,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,Male,NO,40_65,AT,countryside,no,no,None,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,Male,yES,40_65,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,Male,nO,26_39,AT,city,None,no,None,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,Female,yES,40_65,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


# GET DATA FROM API

Some information related to career info table has to be extract from an API. 
In this table, the column "normalized_job_code" is filled with codes that represents different jobs.
With the following code extracts the needed info from the API from the Open Skills Project and stores it in a dictionary.

Let's make a test of connection to the API

In [141]:
#http://api.dataatwork.org/v1/jobs/
base_url = 'http://api.dataatwork.org/v1/jobs/861a9b9151e11362eb3c77ca914172d0'
response = requests.get(base_url)
results = response.json()
results

{'uuid': '861a9b9151e11362eb3c77ca914172d0',
 'title': 'Automatic Data Processing Planner',
 'normalized_job_title': 'automatic data processing planner',
 'parent_uuid': '0148f61d4227497728ce33490843d056'}

A function is created to make a call to the API with every job code in our career table (list_job_codes previously obtained).
The information obtain in each call (a dictionary) is stored in a list. Thus we obtain a list of dictionaries.

In [143]:
def create_api_list(list_job):
    api_list = []
    for x in list_job:
        base_url = f'http://api.dataatwork.org/v1/jobs/{x}'
        response = requests.get(base_url)
        results = response.json()
        api_list.append(results)
    return api_list

In [144]:
api_list = create_api_list(list_job_codes)
print(api_list[5])

{'uuid': 'd71c4164c99a5f9bca773e755a72f40f', 'title': 'Geographic Information Systems Database Administrator (GIS Database Administrator)', 'normalized_job_title': 'geographic information systems database administrator gis database administrator', 'parent_uuid': 'bab6b228ad976cd2624acbcfccb8ad9b'}


The list of dictionaries is converted into a dataframe and then into a full dictionary, for easier access to information.

In [145]:
api_df = pd.DataFrame(api_list)
display(api_df)

,uuid,title,normalized_job_title,parent_uuid
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist,data entry specialist,bf88358c56bb6cbe7eabec38ae333d19
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect,database architect,596226d5f9ef63de41a852826d483d19
...,...,...,...,...
151,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner,analytical data miner,9656fa2dc39e9643c00a45858e0117d0
152,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48
153,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk,data examination clerk,da412504dd7b130414b7bbfa2acd563b
154,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk,data entry clerk,bf88358c56bb6cbe7eabec38ae333d19


In [146]:
jobs_dic = api_df.set_index('uuid').to_dict()['title']

Using the dictionary, the table df_career_info can be completed, replacing the code by the job title.

In [147]:
df_career_info.replace({"normalized_job_code": jobs_dic})

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,Automatic Data Processing Planner
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,Data Coordinator
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,Database Developer
...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,Data Warehouse Developer
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,Database Manager
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,Data Officer


In [148]:
def clean_jobs(table_jobs, api_df):
    for x in table_jobs['normalized_job_code']:
        if x != None:
            
    

SyntaxError: unexpected EOF while parsing (<ipython-input-148-d73523121758>, line 5)

In [180]:
test_dict = {'table1': 1, 'table2': 2, 'table3': 3, 'table4': 4}

In [181]:
def join_tables(test_dict):
    tables_list = list(test_dict.keys())
    print(tables_list)
    tables_dict_list = [f'tables_dict[{x}]' for x in tables_list]
    
    return tables_dict_list

In [182]:
join_tables(test_dict)

['table1', 'table2', 'table3', 'table4']


['tables_dict[table1]',
 'tables_dict[table2]',
 'tables_dict[table3]',
 'tables_dict[table4]']

In [ ]:
table_complete = reduce(lambda left, right: pd.merge(left, right, on='uuid'), tables_list)

In [ ]:
db_dic ={}
for name in db_raw_list:
    print(name)
    db_dic[f'db_{name}'] = pd.read_sql_query("select * FROM {name}".format(name = name), engine)
display(db_dic['db_personal_info'])